# Introduction

In this notebook, we assume that you have already populated your Wikidata (AKA "Method 2") database, which was shown in `notebooks/03-wikidata_kg.ipynb`.  We also will assume that you have run the Cypher queries found in `cypher_queries/method2_queries.cql` to do things like update the node labels to the P31 values, segment out both model and holdback data, and create some basic embeddings.

If you are running this on Google Colab, run the following cell to import the necessary packages.

In [ ]:
!pip install neo4j

In [ ]:
%matplotlib inline

import json
import re
import urllib
from pprint import pprint
import time
from tqdm import tqdm

from neo4j import GraphDatabase

import numpy as np
import pandas as pd

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

import pprint

In [ ]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [ ]:
# If you are using a Docker container for your DB, use the uncommented line.
# conn = Neo4jConnection(uri="bolt://some_ip_address:7687", user="neo4j", pwd="some_password")

uri = ''
user = 'neo4j'
pwd = ''

conn = Neo4jConnection(uri=uri, user=user, pwd=pwd)
conn.query("MATCH (n) RETURN COUNT(n)")

# Embedding visualization

We will begin by looking at whether our embeddings cluster at all in any reasonable way.  We have used a pretty high-dimensional space to create those embeddings though, so we will use t-SNE to do dimensionality reduction for the sake of visualization.  Here we have two categories with 1 = `is_place` and 0 = NOT `is_place`.  We ideally want our embeddings to cluster in some clear fashion to give us some assurances that we are going the right way.

### Note

We have not really done any tuning of our embeddings.  Embedding tuning is an art unto itself and beyond the scope of this demonstration.  However, the result will be, as you will see, that the embeddings don't cluster very well and the resulting ML models don't product super great results.  The following is more intended for demonstration purposes.  In a real-world, deployable application you will obviously want to put much more effort into creating embeddings that work for you.

Also, recall that we have some class imbalance.  There are several ways to deal with it.  For the sake of this demonstration, we will attempt to handle this using some built-in functionality in `scikit-learn`.

In [ ]:
def create_tsne_plot(emb_name='n.n2v_all_nodes', n_components=2, debug=False):
    
    query_string = '''
        MATCH (n:Node)
        RETURN n.name, n.type, n.is_place AS category, {} AS vec
    '''.format(emb_name)
    model_df = pd.DataFrame([dict(_) for _ in conn.query(query_string)])
    
    if debug:
        uniqueValues = model_df['category'].nunique()
        print(uniqueValues)
    
    X_emb = TSNE(n_components=n_components).fit_transform(list(model_df['vec']))
    
    tsne_df = pd.DataFrame(data = {
        'x': [value[0] for value in X_emb],
        'y': [value[1] for value in X_emb],
        'label': model_df['category']
    })
    
    plt.figure(figsize=(16,10))
    s = 30
    ax = sns.scatterplot(
        x='x', y='y',
        palette=sns.color_palette('hls',2),
        data=tsne_df,
        hue='label',
        legend=True, 
        s=500,
        alpha=0.75
    )
    ax.legend(prop={'size': 20})
    plt.xlabel('X Component', fontsize=16)
    plt.ylabel('Y Component', fontsize=16)
    plt.show

    return tsne_df

In [ ]:
tsne_df = create_tsne_plot(emb_name='n.n2v_all_nodes', n_components=2, debug=False)

In [ ]:
tsne_df = create_tsne_plot(emb_name='n.frp_all_nodes', n_components=2, debug=False)

# ML Modeling

There are many ways we could use these embeddings to create a model.  In this example, our goal will be binary classification to predict `is_place` given our different embeddings.  For the sake of simplicity, I am just going to create a support vector machine to demonstrate the general approach.  Obviously you can choose whatever classification model you would like.

### Note

Neo4j outputs its embeddings in a format that is not entirely compatible with `scikit-learn`.  So `create_X` is a helper function that will reformat those embeddings into something suitable.

In [ ]:
def create_X(df2, emb):

    n2v_an_ls = df2[emb].to_list()
    n2v_arr = np.array([np.array(x) for x in n2v_an_ls], dtype=object)

    print(n2v_arr.shape)
    
    return n2v_arr


def modeler(df, emb_name, y_column_name, k_folds=5, model='linear', show_matrix=True):
    
    y = df[y_column_name].fillna(0.0).to_numpy()
    vec_array = create_X(df, emb_name)
    acc_scores = []
    
    pos = np.count_nonzero(y == 1.0)
    neg = y.shape[0] - pos
    print('Number of positive: ', pos, ' Number of negative: ', neg)
    
    for i in range(0, k_folds):
        
        X_train, X_test, y_train, y_test = train_test_split(vec_array, y, test_size=0.25)
        clf = svm.SVC(kernel='linear', class_weight='balanced')
        clf.fit(X_train, y_train)
        pred = clf.predict(X_test)

        acc = accuracy_score(pred, y_test)
        acc_scores.append(acc)        
        
    print('Accuracy scores: ', acc_scores)
    print('Mean accuracy: ', np.mean(acc_scores))
    
    if show_matrix:
        matrix = plot_confusion_matrix(clf, X_test, y_test, cmap=plt.cm.Blues, normalize='true')
        plt.show(matrix)
        plt.show()
        
    return clf

In [ ]:
query_string = '''
    MATCH (n:Node)
    RETURN n.name, n.type, n.is_place AS category, n.n2v_all_nodes AS n2v_vec, n.frp_all_nodes AS frp_vec
'''
model_df = pd.DataFrame([dict(_) for _ in conn.query(query_string)])
model_df.head()

In [ ]:
n2v_clf = modeler(model_df, emb_name='n2v_vec', y_column_name='category')

In [ ]:
frp_clf = modeler(model_df, emb_name='frp_vec', y_column_name='category')

# Observation

So those accuracy scores were really nothing to write home about.  But how do the classifiers generated by the training handle predicting unknown data points?

For this, I am going to choose some nodes that I know to be places just to see what happens.  Some good ones are "city of the United States," "Illinois," and "Oceania."  

Let's just give those a manual try and see what happens.

In [ ]:
def predict_unknown(node_name, emb_name, clf, debug=False):
    
    query_string = "MATCH (n:Node {name: '" + node_name + "'}) RETURN n.name AS name, n." + emb_name + " AS vec"
    
    if debug == True:
        print(query_string)
        print(type(query_string))
        print(emb_name)

    unknown_df = pd.DataFrame([dict(_) for _ in conn.query(query_string)])
    
    vec_array = create_X(unknown_df, emb='vec')
    pred = clf.predict(vec_array)
    print('Predicted Class: ', pred[0])
    
    return

In [ ]:
predict_unknown('city of the United States', 'n2v_all_nodes', clf=n2v_clf, debug=False)

# Result

Disappointing, for sure.  But remember that we have spent no effort in optimizing either the embeddings nor the model.  It would take more time than we have in a 2 hour demonstration to do that.  There are plenty of resources out there to study up on this bit.  

# Next steps

1. Add more data to the graph (always a good idea when you can and time permits!)
2. Optimize the above embeddings (obvious)
3. Try GraphSAGE (advantage: takes into account node properties in addition to looking at random-walk approach)

## Important note

The Graph Data Science library does much more than embeddings!  In particular, I recommend you check out the [node classification](https://neo4j.com/docs/graph-data-science/current/algorithms/ml-models/node-classification/#algorithms-ml-nodeclassification) and [link prediction](https://neo4j.com/docs/graph-data-science/current/algorithms/ml-models/linkprediction/) modeling capabilities.  This small graph might not be able to take much advantage of them, but as the graph gets bigger you might find them to be really helpful!